In [ ]:
Steps:
1) text
2 tokenize 
3 zero padding
4 classification input and ouput ke form mein 
-------------LSTM--------------------------------
1) forget gate
    ft =sig(W (ht-1,xt)+b)
    pointwise operation
   ctf= ct-1 x ft
2) input gate
    ctc=tanh(w (ht-1,xt)+b)
    it=sigmoid(w (ht-1,xt)+b)
    filter
   cell state= ctf + (ctc * it)
3) ouput gate
    ot=sigmoid(w (ht-1,xt)+b)
    ht=ot*tanh(cell state)

In [1]:
text='''I love deep learning.
    LSTMs are powerful models.
    They handle sequences well.'''

In [2]:
sentences = text.strip().split('\n')
print(sentences)

['I love deep learning.', '    LSTMs are powerful models.', '    They handle sequences well.']


In [16]:
vocab_size = len(tokenizer.word_index) + 1


In [3]:
import numpy as np
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.initializers import Zeros

2025-07-19 18:02:44.109865: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752948164.403501      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752948164.507863      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
cleaned_sentences = [s.lower().replace('.', '') for s in sentences]

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_sentences)
sequences = tokenizer.texts_to_sequences(cleaned_sentences)

print(sequences)

[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]


In [17]:
input_sequences = []

for line in cleaned_sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_seq = token_list[:i+1]
        input_sequences.append(n_gram_seq)

In [18]:
max_seq_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

In [19]:
y

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [21]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10))
model.build(input_shape=(None, max_seq_len - 1))
model.add(LSTM(units=3,
               kernel_initializer=Zeros(),
               recurrent_initializer=Zeros(),
               bias_initializer=Zeros()))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 3, 10)          │           130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 3)              │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 13)             │            52 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 350 (1.37 KB)

 Trainable params: 350 (1.37 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
history=model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 2.5649
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.1111 - loss: 2.5642
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.1111 - loss: 2.5634
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.1111 - loss: 2.5626
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.1111 - loss: 2.5618
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.1111 - loss: 2.5611
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.1111 - loss: 2.5603
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.1111 - loss: 2.5595
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.1111 - loss: 2.5587
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.1111 - loss: 2.5579
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.1111 - loss: 2.5570
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.1111 - 

In [23]:
def predict_next_word(seed_text, max_seq_len):
    token_list = tokenizer.texts_to_sequences([seed_text.lower()])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
    return tokenizer.index_word.get(predicted[0], "<unk>")

print(predict_next_word("i", max_seq_len))              # → love
print(predict_next_word("i love", max_seq_len))         # → deep
print(predict_next_word("i love deep", max_seq_len))    # → learning
print(predict_next_word("lstms are", max_seq_len))      # → powerful

learning
learning
learning
learning


# [](http://)--------------------LSTM FROM SCRATCH---------------------------

In [24]:
import numpy as np

In [25]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [26]:
def tanh(x):
    return np.tanh(x)

In [31]:
def init_lstm(input_dim,hidden_dim=3):
    concat_dim = input_dim + hidden_dim
    def init_random():
        return np.random.rand(hidden_dim, concat_dim), np.random.rand(hidden_dim, 1)

    W_f, b_f = init_random()
    W_i, b_i = init_random()
    W_c, b_c = init_random()
    W_o, b_o = init_random()

    return {
        'W_f': W_f,'b_f': b_f,
        'W_i': W_i,'b_i': b_i,
        'W_c': W_c,'b_c': b_c,
        'W_o': W_o,'b_o': b_o
    }

In [32]:
init_lstm(max_seq_len - 1)

{'W_f': array([[0.27554774, 0.05114868, 0.52193946, 0.28276416, 0.11392216,
         0.37469723],
        [0.24051152, 0.31213082, 0.51891327, 0.76442642, 0.77522123,
         0.40098425],
        [0.35046692, 0.13693592, 0.67506659, 0.92354216, 0.05454694,
         0.49245773]]),
 'b_f': array([[0.7735911 ],
        [0.34690299],
        [0.9685452 ]]),
 'W_i': array([[0.26575617, 0.17887051, 0.84178553, 0.70874152, 0.91728855,
         0.05090971],
        [0.33026116, 0.06929789, 0.57393812, 0.56237187, 0.76901633,
         0.81841684],
        [0.2867218 , 0.13058046, 0.59663894, 0.1896095 , 0.04958516,
         0.59567737]]),
 'b_i': array([[0.33762692],
        [0.67607138],
        [0.29414559]]),
 'W_c': array([[0.64458964, 0.0392135 , 0.13251645, 0.10971534, 0.34701961,
         0.03107535],
        [0.57513309, 0.38415681, 0.82995961, 0.40756457, 0.81730976,
         0.05544312],
        [0.34066872, 0.04075343, 0.61073153, 0.59297604, 0.36532172,
         0.03301788]]),
 'b_

In [33]:
def lstm_step(params, x_t, h_prev, C_prev):
    W_f, b_f = params['W_f'], params['b_f']
    W_i, b_i = params['W_i'], params['b_i']
    W_c, b_c = params['W_c'], params['b_c']
    W_o, b_o = params['W_o'], params['b_o']

    concat = np.vstack((x_t, h_prev))
    
    ft=sigmoid(np.dot(W_f,concat)+b_f)
    forget=ft*C_prev

    Cct=tanh(np.dot(W_c,concat)+b_c)
    it=sigmoid(np.dot(W_i,concat)+b_i)

    filters=it*Cct

    Ct=forget+filters

    ot=sigmoid(np.dot(W_o,concat)+b_o)

    ht=ot*(tanh(Ct))

    return ht,Ct

In [34]:
def lstm(param,input_dim,hidden_dim=3):
    ht = np.zeros((hidden_dim, 1))
    Ct= np.zeros((hidden_dim, 1))
    for x_t in inputs:
        ht, Ct = lstm_step(params, x_t, ht, Ct)
    return ht

In [35]:
def softmax(x):
    ex = np.exp(x - np.max(x))
    return ex / ex.sum(axis=0)

In [40]:
def predict_next_word(ht, vocab_size):
    W_y = np.random.rand(vocab_size, ht.shape[0])  
    b_y = np.random.rand(vocab_size, 1)
    logits = np.dot(W_y, ht) + b_y
    probs = softmax(logits)
    return np.argmax(probs), probs